## MNIST digits classification with mini_torch 

In [1]:
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from mini_torch import Tensor, Linear, sigmoid, SGD, Module, cross_entropy, Adam

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



class NeuralNet(Module):
    def __init__(self):
        super().__init__()
        self.linear1 = Linear(28 * 28, 128)
        self.linear2 = Linear(128, 10)

    def forward(self, x):
        x = x.reshape(x.shape[0], -1)  # Flatten the input
        x = sigmoid(self.linear1(x))
        return self.linear2(x)

model = NeuralNet()
optimizer = SGD(model.parameters(), lr=0.01)
optimizer = Adam(model.parameters(), lr=0.01)

/Users/homefolder/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/homefolder/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c1017RegisterOperatorsD1Ev)
  Referenced from: '/Users/homefolder/miniconda3/lib/python3.10/site-packages/torchvision/image.so'
  Expected in: '/Users/homefolder/miniconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        # Forward pass
        outputs = model(Tensor(images))
        loss = cross_entropy(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        running_loss += loss.data

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")


/Users/homefolder/Desktop/mini_torch/mini_torch/functions.py:10: RuntimeWarning: overflow encountered in exp
  out_data = 1 / (1 + np.exp(-x.data))


Epoch 1/5, Loss: 0.6102032867480697
Epoch 2/5, Loss: 0.40619478485922317
Epoch 3/5, Loss: 0.38802448594207956
Epoch 4/5, Loss: 0.35514134510176015
Epoch 5/5, Loss: 0.33754396622877386


In [3]:
# Evaluation loop
all_preds = []
all_labels = []

for images, labels in test_loader:
    images = Tensor(images)
    outputs = model(images)
    preds = np.argmax(outputs.data, axis=1) 
    all_preds.extend(preds)
    all_labels.extend(labels)

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 91.15%
